***Innomatics Research Labs | Advanced GenAI Internship | Entrance Test***




Datasets merging

In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
# Create SQLite DB
conn = sqlite3.connect("restaurants.db")

# Read SQL file
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

# Execute SQL
conn.executescript(sql_script)

# Load table into pandas
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)


,name
0,restaurants


In [6]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


***MCQS***

Which city has the highest total revenue (total_amount) from Gold members?  

In [8]:
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum() \
.sort_values(ascending=False).head(1)


,total_amount
city,
Chennai,1080909.79


Which cuisine has the highest average order value across all orders?


In [9]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False).head(1)


,total_amount
cuisine,
Mexican,808.021344


How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [29]:
final_df.groupby("user_id")["total_amount"] \
    .sum() \
    .gt(1000) \
    .sum()



np.int64(2544)

Which restaurant rating range generated the highest total revenue?


In [12]:
final_df.assign(
    rating_range=(final_df["rating"].astype(float).round(1))
).groupby("rating_range")["total_amount"].sum().sort_values(ascending=False).head(1)


,total_amount
rating_range,
4.8,657707.71


Among Gold members, which city has the highest average order value?


In [14]:
final_df[final_df["membership"] == "Gold"] \
.groupby("city")["total_amount"].mean() \
.sort_values(ascending=False).head(1)


,total_amount
city,
Chennai,808.45908


Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [15]:
final_df.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values(
    by=["distinct_restaurants", "total_revenue"],
    ascending=[True, False]
).head(1)


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65


What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [16]:
round(
    (final_df[final_df["membership"] == "Gold"].shape[0] / final_df.shape[0]) * 100
)


50

Which restaurant has the highest average order value but less than 20 total orders?

In [28]:
final_df.groupby(["restaurant_id", "restaurant_name_x"]).agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
).query("total_orders < 20") \
.sort_values("avg_order_value", ascending=False) \
.head(1)



,,avg_order_value,total_orders
restaurant_id,restaurant_name_x,,
294,Hotel Dhaba Multicuisine,1040.222308,13


Which combination contributes the highest revenue?



In [25]:
final_df[
    (
        (final_df["membership"] == "Gold") & (final_df["cuisine"] == "Indian")
    ) | (
        (final_df["membership"] == "Gold") & (final_df["cuisine"] == "Italian")
    ) | (
        (final_df["membership"] == "Regular") & (final_df["cuisine"] == "Indian")
    ) | (
        (final_df["membership"] == "Regular") & (final_df["cuisine"] == "Chinese")
    )
].groupby(["membership", "cuisine"])["total_amount"] \
 .sum() \
 .sort_values(ascending=False) \
 .head(1)


,,total_amount
membership,cuisine,
Gold,Italian,1005779.05


During which quarter of the year is the total revenue highest?


In [27]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], errors="coerce")

final_df.assign(
    quarter=final_df["order_date"].dt.quarter
).groupby("quarter")["total_amount"] \
 .sum() \
 .sort_values(ascending=False) \
 .head(1)



/tmp/ipython-input-1414396507.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"], errors="coerce")


,total_amount
quarter,
3,2037385.1


***numericals***

How many total orders were placed by users with Gold membership?

In [31]:
final_df.loc[final_df["membership"] == "Gold", "order_id"] \
    .count()


np.int64(4987)

What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [32]:
round(
    final_df.loc[final_df["city"] == "Hyderabad", "total_amount"].sum()
)


1889367

How many distinct users placed at least one order?

In [33]:
final_df["user_id"].nunique()


2883

What is the average order value (rounded to 2 decimals) for Gold members?


In [35]:
round(
    final_df.loc[final_df["membership"] == "Gold", "total_amount"].mean(),
    2
)


np.float64(797.15)

How many orders were placed for restaurants with rating ≥ 4.5?

In [36]:
final_df.loc[final_df["rating"] >= 4.5, "order_id"].count()


np.int64(3374)

How many orders were placed in the top revenue city among Gold members only?

In [38]:
gold_df = final_df[final_df["membership"] == "Gold"]

top_city = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

gold_df.loc[gold_df["city"] == top_city, "order_id"].count()


np.int64(1337)

***fill in the blanks***

In [39]:
final_df.shape[0]


10000

In [41]:
final_df["user_id"].nunique()


2883

In [42]:
final_df.to_csv("final_merged_dataset.csv", index=False)

from google.colab import files
files.download("final_merged_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>